In [1]:
import cv2 as cv
import numpy as np
import os

In [2]:
"""
Draw epipolar lines in img2 corresponding to pts1 in img1, and vice versa.
"""
def draw_epilines(img1, img2, pts1, pts2, F):    
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]

    # Compute epilines in img2 for points in img1: l2 = F * x1
    lines2 = cv.computeCorrespondEpilines(pts1.reshape(-1, 1, 2), 1, F).reshape(-1, 3)
    img2_lines = img2.copy()

    for (a, b, c), p2 in zip(lines2, pts2):
        # line: a*x + b*y + c = 0
        x0, y0 = 0, int(-c / b) if abs(b) > 1e-9 else 0
        x1, y1 = w2, int(-(c + a * w2) / b) if abs(b) > 1e-9 else h2
        cv.line(img2_lines, (x0, y0), (x1, y1), (0, 255, 0), 1)
        cv.circle(img2_lines, tuple(p2.astype(int)), 4, (0, 0, 255), -1)

    # Compute epilines in img1 for points in img2: l1 = F^T * x2
    lines1 = cv.computeCorrespondEpilines(pts2.reshape(-1, 1, 2), 2, F).reshape(-1, 3)
    img1_lines = img1.copy()

    for (a, b, c), p1 in zip(lines1, pts1):
        x0, y0 = 0, int(-c / b) if abs(b) > 1e-9 else 0
        x1, y1 = w1, int(-(c + a * w1) / b) if abs(b) > 1e-9 else h1
        cv.line(img1_lines, (x0, y0), (x1, y1), (0, 255, 0), 1)
        cv.circle(img1_lines, tuple(p1.astype(int)), 4, (0, 0, 255), -1)

    return img1_lines, img2_lines

In [3]:
def main(img_path1, img_path2, max_vis=30):
    img1 = cv.imread(img_path1)
    img2 = cv.imread(img_path2)
    if img1 is None or img2 is None:
        raise FileNotFoundError("Could not read one of the images.")

    gray1 = cv.cvtColor(img1, cv.COLOR_BGR2GRAY)
    gray2 = cv.cvtColor(img2, cv.COLOR_BGR2GRAY)

    # ORB features (fast, free)
    orb = cv.ORB_create(nfeatures=4000)
    k1, d1 = orb.detectAndCompute(gray1, None)
    k2, d2 = orb.detectAndCompute(gray2, None)

    if d1 is None or d2 is None:
        raise RuntimeError("No descriptors found. Try different images.")

    bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=False)
    matches = bf.knnMatch(d1, d2, k=2)

    # Lowe ratio test
    good = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good.append(m)

    if len(good) < 12:
        raise RuntimeError(f"Not enough good matches ({len(good)}). Try more textured images.")

    pts1 = np.float32([k1[m.queryIdx].pt for m in good])
    pts2 = np.float32([k2[m.trainIdx].pt for m in good])

    # Robust F with RANSAC
    F, mask = cv.findFundamentalMat(pts1, pts2, method=cv.FM_RANSAC, ransacReprojThreshold=1.0, confidence=0.99)
    if F is None:
        raise RuntimeError("Fundamental matrix estimation failed.")

    inliers1 = pts1[mask.ravel() == 1]
    inliers2 = pts2[mask.ravel() == 1]
    print(f"Good matches: {len(good)} | Inliers: {len(inliers1)}")
    print("F:\n", F)

    # Visualize a subset of inlier correspondences with epipolar lines
    n = min(max_vis, len(inliers1))
    idx = np.random.choice(len(inliers1), n, replace=False)
    vis1, vis2 = draw_epilines(img1, img2, inliers1[idx], inliers2[idx], F)

    cv.imshow("Image 1 - epilines from Image 2 points", vis1)
    cv.imshow("Image 2 - epilines from Image 1 points", vis2)

    # Also show matched features (inliers only)
    inlier_matches = [m for m, keep in zip(good, mask.ravel()) if keep]
    match_vis = cv.drawMatches(img1, k1, img2, k2, inlier_matches[:200], None, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    cv.imshow("Inlier matches (first 200)", match_vis)

    cv.waitKey(0)
    cv.destroyAllWindows()

In [4]:
img_dir = r'D:\Python things\middle-ml-cv-roadmap\data\raw'
img_1 = 'img_example_11.jpg'
img_2 = 'img_example_12.jpg'
img_1_path = os.path.join(img_dir, img_1)
img_2_path = os.path.join(img_dir, img_2)

In [5]:
main(img_1_path, img_2_path)

Good matches: 298 | Inliers: 123
F:
 [[-2.85667211e-06 -2.17996468e-05  1.61207538e-02]
 [ 3.04923486e-05 -4.35045063e-07 -3.60593117e-02]
 [-1.74560433e-02  3.39166362e-02  1.00000000e+00]]
